In [20]:
import codecs
import networkx as nx
import pickle
from os import listdir
from os.path import isfile, join
from progressbar import printProgressBar

In [8]:
input_dir = 'pairs'
input_files = dict()
input_files['converts-formate-V4-avecsem-utf8-lid-newtagset-entete-new-rid.csv'] = 'A'
input_files['demonette1-formate-V4-avecsem-utf8-lid-newtagset-entete-new-rid.csv'] = 'B'
input_files['denomXaire-formate-V4-avecsem-utf8-lid-newtagset-entete-new-rid.csv'] = 'C'
input_files['denomXal-formate-V4-avecsem-utf8-lid-newtagset-entete-new-rid.csv'] = 'D'
input_files['denomXique-formate-V4-avecsem-utf8-lid-newtagset-entete-new-rid.csv'] = 'E'
input_files['dimocXaie-formate-V4-avecsem-utf8-lid-newtagset-entete-new-rid.csv'] = 'F'
input_files['dimocXat-formate-V4-avecsem-utf8-lid-newtagset-entete-new-rid.csv'] = 'G'
input_files['dimocXet-formate-V4-avecsem-utf8-lid-newtagset-entete-new-rid.csv'] = 'H'
input_files['dimocXier-formate-V4-avecsem-utf8-lid-newtagset-entete-new-rid.csv'] = 'I'
input_files['derifde1X-formate-V4-nosem-utf8-lid-newtagset-entete-new-rid.csv'] = 'J'
input_files['derifXable-formate-V4-nosem-utf8-lid-newtagset-entete-new-rid.csv'] = 'K'

# column number
graph_1 = 3
graph_2 = 6
cat_1 = 8
cat_2 = 10
cstr_1 = 14
cstr_2 = 17
complexite = 19
orientation = 21

In [12]:
header = ''
G = nx.DiGraph()
for input_file in input_files:
    with codecs.open(join(input_dir, input_file), 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f):
            if line_num == 0:
                header = line.replace('\n','')
            elif line_num >= 2:
                elements = line.replace('\n','').split('\t')
                G.add_node(elements[graph_1] + '_' + elements[cat_1], label=elements[cat_1])
                G.add_node(elements[graph_2] + '_' + elements[cat_2], label=elements[cat_2])
                if elements[orientation] == 'as2de' or elements[orientation] == 'as2des':
                    edge_type = elements[complexite] + '_' + elements[cstr_1] + '-' + elements[cstr_2]
                    G.add_edge(elements[graph_1] + '_' + elements[cat_1], elements[graph_2] + '_' + elements[cat_2], label=edge_type)
                elif elements[orientation] == 'de2as' or elements[orientation] == 'des2as':
                    edge_type = elements[complexite] + '_' + elements[cstr_2] + '-' + elements[cstr_1]
                    G.add_edge(elements[graph_2] + '_' + elements[cat_2], elements[graph_1] + '_' + elements[cat_1], label=edge_type)
                else:
                    sorted_cstr = sorted([elements[cstr_1], elements[cstr_2]])
                    edge_type =  elements[complexite] + '_' + elements[orientation] + '_' + sorted_cstr[0] + '-' + sorted_cstr[1]
                    G.add_edge(elements[graph_1] + '_' + elements[cat_1], elements[graph_2] + '_' + elements[cat_2], label=edge_type)
                    G.add_edge(elements[graph_2] + '_' + elements[cat_2], elements[graph_1] + '_' + elements[cat_1], label=edge_type)

In [21]:
conn_comps = list(nx.weakly_connected_components(G))
number_of_families = len(conn_comps)
number_of_families

13179

In [26]:
checked = list()
H = nx.Graph() # graph of graphs
for fam1 in range(0, number_of_families):
    if fam1 in checked:
        continue
    for fam2 in range(fam1 + 1, number_of_families):
        if fam2 in checked:
            continue
        G1 = nx.subgraph(G, conn_comps[fam1])
        G2 = nx.subgraph(G, conn_comps[fam2])
        if nx.is_isomorphic(G1, G2, node_match=lambda v1,v2: v1['label'] == v2['label'],\
                            edge_match=lambda e1,e2: e1['label'] == e2['label']):
            checked.append(fam2)
            H.add_edge(fam1, fam2)
    printProgressBar(fam1 + 1, number_of_families, prefix = 'Progress:', suffix = 'Complete', length = 50)
H_conn_comps = list(nx.connected_components(H))
print(len(H_conn_comps))

KeyboardInterrupt: 

In [ ]:
H_conn_comps = [c for c in sorted(nx.connected_components(G), key=len, reverse=True)]
H_conn_comps

In [8]:
output_folder = 'families'
lexeme_dict = dict()
f_summary = codecs.open('summary_of_families.txt', 'w+', encoding='utf-8')
f_summary.write('family_id\tnumber_of_lexemes\tnumber_of_relations\tlexemes\n')
for family_id, lexeme_family in enumerate(conn_comps):
    first_member = sorted(list(lexeme_family))[0] # first alphabetically
    H = nx.subgraph(G, lexeme_family)
    f_summary.write('F' + str(family_id).rjust(5, '0') + '\t' + str(len(lexeme_family)) + '\t' + str(len(H.edges)) + '\t' + str(lexeme_family) + '\n')
    for lexeme in lexeme_family:
        filename = 'F' + str(family_id).rjust(5, '0') + ' ' + first_member + '.txt'
        lexeme_dict[lexeme] = filename
        f_out = codecs.open(join(output_folder, filename), 'w+', encoding='utf-8')
        f_out.write(header + '\tfichier_origine' + '\n\n')
        f_out.close()
f_summary.close()
#lexeme_dict['pommier']

In [9]:
for input_file in input_files:
    with codecs.open(join(input_dir, input_file), 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f):
            if line_num < 2:
                continue
            elements = line.split('\t')
            lexeme1 = elements[graph_1] + '_' + elements[cat_1]
            output_filename = lexeme_dict[lexeme1]
            f_out = codecs.open(join(output_folder, output_filename), 'a+', encoding='utf-8')
            f_out.write(line.strip('\n') + '\t' + input_files[input_file] + '\n')
            f_out.close()

In [ ]:
graph_file = open('G.p', 'wb')
dict_file = open('lexeme_dict.p', 'wb')
pickle.dump(G, graph_file)
pickle.dump(lexeme_dict, dict_file)
graph_file.close()
dict_file.close()